# Example usage of [stable-dreamfusion](https://github.com/ashawkey/stable-dreamfusion)

### Check the machine

In [ ]:
! nvidia-smi

Tue Apr  4 02:22:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Setup

In [ ]:
#@title install dependencies
! git clone https://github.com/ashawkey/stable-dreamfusion.git

%cd stable-dreamfusion

# fix the commit
! git reset --hard e496e35df6e9b67784c3b24504cd61a2e7c19a0d

# install requirements
! pip install -r requirements.txt
! pip install git+https://github.com/NVlabs/nvdiffrast/

# install CUDA extensions (takes about 8 minutes!)
! pip install ./raymarching
! pip install ./shencoder
! pip install ./freqencoder
! pip install ./gridencoder

fatal: destination path 'stable-dreamfusion' already exists and is not an empty directory.
/content/stable-dreamfusion
HEAD is now at e496e35 Merge pull request #184 from MathieuTuli/main
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVlabs/nvdiffrast/ to /tmp/pip-req-build-smicjb95
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/nvdiffrast/ /tmp/pip-req-build-smicjb95
  Resolved https://github.com/NVlabs/nvdiffrast/ to commit fad71a4ad3dc8530a4eeaeb55a65b2e8c127d69d
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./raymarching
  Preparing metadata (setup.py) ... done
  Created wheel for raymarching: filename=raymarching-0.0.0-cp39-cp39-linux_x86_64.whl size=2823452 sha256=a

### Training & Testing
* The default 5000 training steps take around 30 minutes to finish. A larger `Training_iters` usually leads to better results.
* If CUDA OOM, try to decrease `Max_steps` and `Training_nerf_resolution`.
* If the NeRF fails to learn anything (empty scene, only background), try to decrease `Lambda_entropy` which regularizes the learned opacity.

In [ ]:
#@markdown ####**Training Settings:**
Prompt_text = "a DSLR photo of a delicious hamburger" #@param {type: 'string'}
Training_iters = 5000 #@param {type: 'integer'}
Learning_rate = 1e-3 #@param {type: 'number'}
Training_nerf_resolution = 64  #@param {type: 'integer'}
# CUDA_ray = True #@param {type: 'boolean'}
# View_dependent_prompt = True #@param {type: 'boolean'}
# FP16 = True #@param {type: 'boolean'}
Seed = 0 #@param {type: 'integer'}
Lambda_entropy = 1e-4 #@param {type: 'number'}
Max_steps = 512 #@param {type: 'number'}
Checkpoint = 'latest' #@param {type: 'string'}

#@markdown ---

#@markdown ####**Output Settings:**
Workspace = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

# processings
Prompt_text = "'" + Prompt_text + "'"

In [ ]:
#@title start training
import torch
torch.cuda.empty_cache()

%run main.py -O --text {Prompt_text} --workspace {Workspace} --iters {Training_iters} --lr {Learning_rate} --w {Training_nerf_resolution} --h {Training_nerf_resolution} --seed {Seed} --lambda_entropy {Lambda_entropy} --ckpt {Checkpoint} --save_mesh --max_steps {Max_steps}

Namespace(text='a DSLR photo of a delicious hamburger', negative='', O=True, O2=False, test=False, eval_interval=1, workspace='trial', guidance='stable-diffusion', seed=0, save_mesh=True, mcubes_resolution=256, decimate_target=100000.0, iters=5000, lr=0.001, warm_iters=500, min_lr=0.0001, ckpt='latest', cuda_ray=True, taichi_ray=False, max_steps=512, num_steps=64, upsample_steps=32, update_extra_interval=16, max_ray_batch=4096, warmup_iters=2000, jitter_pose=False, uniform_sphere_rate=0.5, bg_radius=1.4, density_activation='softplus', density_thresh=0.1, blob_density=10, blob_radius=0.5, backbone='grid', optim='adan', sd_version='2.1', hf_key=None, fp16=True, vram_O=False, w=64, h=64, bound=1, dt_gamma=0, min_near=0.1, radius_range=[1.0, 1.5], fovy_range=[40, 70], dir_text=True, suppress_face=False, angle_overhead=30, angle_front=60, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, lambda_tv=0, gui=False, W=800, H=800, radius=3, fovy=60, light_theta=60, light_phi=0, max_spp

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


[INFO] loaded stable diffusion!


[INFO] Cmdline: main.py -O --text a DSLR photo of a delicious hamburger --workspace trial --iters 5000 --lr 0.001 
--w 64 --h 64 --seed 0 --lambda_entropy 0.0001 --ckpt latest --save_mesh --max_steps 512

...

[INFO] load at epoch 50, global step 5000

[INFO] loaded optimizer.

[INFO] loaded scheduler.

[INFO] loaded scaler.

[INFO] training takes 0.0000 minutes.

In [ ]:
#@markdown ####**Testing Settings:**

Workspace_test = "trial" #@param{type: 'string'}
# Save_mesh = True #@param {type: 'boolean'}

In [ ]:
#@title testing
%run main.py -O --test --workspace {Workspace_test} --save_mesh

Namespace(text=None, negative='', O=True, O2=False, test=True, eval_interval=1, workspace='trial', guidance='stable-diffusion', seed=0, save_mesh=True, mcubes_resolution=256, decimate_target=100000.0, iters=10000, lr=0.001, warm_iters=500, min_lr=0.0001, ckpt='latest', cuda_ray=True, taichi_ray=False, max_steps=1024, num_steps=64, upsample_steps=32, update_extra_interval=16, max_ray_batch=4096, warmup_iters=2000, jitter_pose=False, uniform_sphere_rate=0.5, bg_radius=1.4, density_activation='softplus', density_thresh=0.1, blob_density=10, blob_radius=0.5, backbone='grid', optim='adan', sd_version='2.1', hf_key=None, fp16=True, vram_O=False, w=64, h=64, bound=1, dt_gamma=0, min_near=0.1, radius_range=[1.0, 1.5], fovy_range=[40, 70], dir_text=True, suppress_face=False, angle_overhead=30, angle_front=60, lambda_entropy=0.0001, lambda_opacity=0, lambda_orient=0.01, lambda_tv=0, gui=False, W=800, H=800, radius=3, fovy=60, light_theta=60, light_phi=0, max_spp=1)
NeRFNetwork(
  (encoder): Grid

[INFO] Cmdline: main.py -O --test --workspace trial --save_mesh

[INFO] Trainer: df | 2023-04-04_02-32-10 | cuda | fp16 | trial

[INFO] #parameters: 12241607

[INFO] Loading latest checkpoint ...

...

==> Finished Test.

100% 100/100 [00:18<00:00,  5.53it/s]


==> Saving mesh to trial/mesh

[INFO] marching cubes thresh: 2.5 (0.0 ~ 80.81651306152344)
[INFO] mesh cleaning: (47856, 3) --> (47088, 3), (95644, 3) --> (94164, 3)
[INFO] running xatlas to unwrap UVs for mesh: v=(47088, 3) f=(94164, 3)
[INFO] writing obj mesh to trial/mesh/mesh.obj
[INFO] writing vertices (47088, 3)
[INFO] writing vertices texture coords (71971, 2)
[INFO] writing faces (94164, 3)


==> Finished saving mesh.

### Display results
* RGB and Depth video are located at `{Workspace}/results/*.mp4`
* Mesh is under `{Workspace}/mesh/`, you could see three files named `mesh.obj`, `mesh.mtl`, and `albedo.png`.

In [ ]:
#@title display RGB video
import os
import glob
from IPython.display import HTML
from base64 import b64encode

def get_latest_file(path):
  dir_list = glob.glob(path)
  dir_list.sort(key=lambda x: os.path.getmtime(x))
  return dir_list[-1]

def show_video(video_path, video_width = 600):

  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

rgb_video = get_latest_file(os.path.join(Workspace, 'results', '*_rgb.mp4'))
show_video(rgb_video)